# Quantum Circuit Simulator
### By Matt Wright
#### 2021-02-15

In [1]:
import numpy as np

In [2]:
def build_u3(params):
    '''Contructs a U3 gate unitary matrix based on input parameters.
    
    Parameters
    ----------
    params : dict
        Parameter name-value pairs. Must contain numeric values for the feilds `theta`,
        `phi`, and `lambda`.
        
    Returns
    -------
    np.ndarray
        U3 unitary calculated with the provided parameters.
    '''
    
    return np.array([[np.cos(params['theta']/2), -np.exp(1j * params['lambda']) * np.sin(params['theta']/2)], 
             [np.exp(1j * params['phi']) * np.sin(params['theta']/2), np.exp(1j * params['lambda'] + 1j * params['phi']) * np.cos(params['theta']/2)]])

In [3]:
# Define unitary representations of gates:
UNITARIES = {
    'x' : [[0, 1], [1, 0]],
    'z' : [[1, 0], [0, -1]],
    'y' : [[0, -1j], [1j, 0]],
    'h' : [[0.5**0.5, 0.5**0.5], [0.5**0.5, -0.5**0.5]],
    'cx' : [[0, 1], [1, 0]],  # unitary for X since control logic is implemented in get_operator()
    'u3': build_u3,
}


def get_ground_state(num_qubits):
    '''Builds a vector representation of the ground state of the "QPU".
    
    Parameters
    ----------
    num_qubits : int
        Number of qubits in the desired QPU.
    
    Returns
    -------
    numpy.ndarray
        Vector of length 2**num_qubits with all zero elements except for the first element.
    '''
    
    state_vector = np.zeros(2**num_qubits)
    state_vector[0] = 1
    return state_vector


def get_operator(total_qubits, gate_unitary, target_qubits, params=None):
    '''Builds a matrix operator out of the gate or unitary provided to be
    applied to the state vector of the QPU.
    
    Parameters
    ----------
    total_qubits : int
        Number of qubits in the QPU.
    gate_unitary : list or str
        This argument is either the unitary matrix or the string representation of the
        unitary (i.e. "h" for Hadamard).
    target_qubits : list[int]
        The indices of the qubit(s) to apply the operator on.
    params : dict
        Has parameter name (i.e. `theta`) and numerical parameter value pairs. Used when
        building parameterized circuits.
    
    Returns
    -------
    numpy.ndarray
        Matrix of size 2**total_qubits x 2**total_qubits which can be applied on the
        state vector of the QPU to operate on the target qubit(s).
    '''
    
    # Get unitary matrix
    if type(gate_unitary) == list:
        U = gate_unitary
    elif gate_unitary in UNITARIES:
        if params is not None:
            U = UNITARIES[gate_unitary](params)
        else:
            U = UNITARIES[gate_unitary]
    else:
        raise Exception('Invalid gate/unitary provided.')
        
    if len(target_qubits) == 0:
        raise Exception('No target qubits provided')
    
    elif len(target_qubits) == 1:  # 1-qubit gate
        I = np.identity(2)
        if target_qubits[0] == 0:
            operator = U
        else:
            operator = I
            
        for i in range(1, total_qubits):
            if i == target_qubits[0]:
                operator = np.kron(operator, U)
            else:
                operator = np.kron(operator, I)
        
        return operator
    
    elif len(target_qubits) == 2:  # Assuming controlled, 2-qubit gate
        # Define|0><0| and |1><1| projectors
        P0x0 = [[1, 0],
                [0, 0]]
        P1x1 = [[0, 0],
                [0, 1]]
        
        I = np.identity(2)
        
        control_qubit = target_qubits[0]
        target_qubit = target_qubits[1]
        
        if control_qubit >= total_qubits or target_qubit >= total_qubits:
            raise Exception('Invalid target qubits provided. Must be in [0, total_qubits-1]')
        
        # Initialize first element of |0> and |1> basis operators
        if control_qubit == 0:
            operator_0 = P0x0
            operator_1 = P1x1
        elif target_qubit == 0:
            operator_0 = I
            operator_1 = U
        else:
            operator_0 = I
            operator_1 = I
            
        for i in range(1, total_qubits):
            if i == control_qubit:
                operator_0 = np.kron(operator_0, P0x0)
                operator_1 = np.kron(operator_1, P1x1)
            elif i == target_qubit:
                operator_0 = np.kron(operator_0, I)
                operator_1 = np.kron(operator_1, U)
            else:
                operator_0 = np.kron(operator_0, I)
                operator_1 = np.kron(operator_1, I)
                
        return operator_0 + operator_1 
    
    else:
        raise Exception('Too many elements in target_qubits. Only support 1 and 2 qubit gates currently.')
        

def get_param_values(instruction, global_params):
    '''Converts global parameter names to the numeric value.

    Parameters
    ----------
    instruction : dict
        Qubit gate description object.
    global_params : dict
        Global parameter name-value pairs (i.e. `{ "global_1" : 3.1415 }`).

    Returns
    -------
    dict
        Gate parameter name-value pirs (i.e. `{ "theta" : 3.1415 }`).
    '''

    params = None
    if 'params' in instruction:
            params = instruction['params'].copy()
            if global_params is not None:
                for param_name, val in params.items():
                    if val in global_params:
                        params[param_name] = global_params[val]        
    return params


def run_program(state_vector, program, global_params=None):
    '''Builds and operates the circuit elements on the state vector of the QPU.
    
    Parameters
    ----------
    state_vector : list
        State vector of the QPU to be operated on.
    program : list[dict]
        Defines the desired quantum circuit. Elements are applied in order of occurance and should
        define the gate/unitary, the target qubit(s), and parameter values if using a parametric gate.
        For example: `{"unitary": [[0.70710678, 0.70710678], [0.70710678, -0.70710678]], 
        "target": [0]}` or `{"gate": "h", "target": [0]}`.
    global_params : dict
        Global parameter name (i.e. `"global_1"`) and numeric value pairs.
    
    Returns
    -------
    numpy.ndarray
        State vector resulting from its evolution through the circuit.
    '''
    
    if 'unitary' in program[0]:
        gate_unitary_key = 'unitary'
    elif 'gate' in program[0]:
        gate_unitary_key = 'gate'
    else:
        raise Exception('Invalid circuit structure. Must define either "unitary" or "gate" fields in circuit.')
    
    num_qubits = int(np.log2(len(state_vector)))
    for instruction in program:
        unitary = instruction[gate_unitary_key]
        target_qubits = instruction['target']
        params = get_param_values(instruction, global_params)
                    
        O = get_operator(num_qubits, unitary, target_qubits, params)
        state_vector = np.dot(O, state_vector)
        
    return state_vector


def measure_all(state_vector):
    '''Randomly collapses the state vector of the QPU based on the amplitudes of each individual state.
    
    Parameters
    ----------
    state : list
        State vector of the QPU to be measured.
        
    Returns
    -------
    str
        Binary string of the collapsed qubit state in big endian form.
    '''
    
    rand_num = np.random.rand()  # Random number uniformly picked form the range of [0,1)
    sum_probs = 0
    
    # Sum the magnitudes of each state's amplitude until it exeeds the generates random number 
    for idx, val in enumerate(state_vector):
        sum_probs += abs(val**2)
        if sum_probs >= rand_num:
            binary_idx = '{0:b}'.format(idx)
            break
    
    num_qubits = int(np.log2(len(state_vector)))
    if len(binary_idx) < num_qubits:
        # Pad binary state with leading zeros if needed
        binary_idx = '0' * (num_qubits-len(binary_idx)) + binary_idx
        
    return binary_idx


def get_counts(state_vector, num_shots=1024):
    '''Measures the state of the QPU multiple times.
    
    Parameters
    ----------
    state_vector : list
        State vector of the QPU to be measured.
    num_shots : int (Default is `1024`)
        Number of attempts to measure the state.
        
    Returns
    -------
    dict
        Dictionary states that occured and their corresponding counts.
    '''

    if num_shots < 0:
        raise Exception('Invalid `num_shots` provided, must be a positive integer.')
    
    result = {}
    for i in range(num_shots):
        idx = measure_all(state_vector)
        if idx not in result:
            result[idx] = 1
        else:
            result[idx] += 1
    return result

## Example 1: 3-Qubit GHZ State

In [4]:
total_qubits = 3

my_circuit = [
    { "gate": "h", "target": [0] },
    { "gate": "cx", "target": [0, 1] },
    { "gate": "cx", "target": [0, 2] }
]

# Create "quantum computer" with 3 qubits
my_qpu = get_ground_state(total_qubits)

# Run circuit
final_state = run_program(my_qpu, my_circuit)

# Read results
counts = get_counts(final_state, 1000)

print(counts)

{'000': 513, '111': 487}


## Example 2: U3 Parametric Gate

In [5]:
total_qubits = 2

my_circuit = [
    { "gate": "u3", "params": { "theta": 3.1415, "phi": 1.5708, "lambda": -3.1415 }, "target": [0] }
]

# Create "quantum computer" with 3 qubits
my_qpu = get_ground_state(total_qubits)

# Run circuit
final_state = run_program(my_qpu, my_circuit)

print(np.round(final_state))

[ 0.+0.j  0.+0.j -0.+1.j  0.+0.j]


## Example 3: Parametric Gate with Global Variables

In [6]:
total_qubits = 2

my_circuit = [
    { "gate": "u3", "params": { "theta": "global_1", "phi": "global_2", "lambda": -3.1415 }, "target": [0] }
]

# Create "quantum computer" with 3 qubits
my_qpu = get_ground_state(total_qubits)

# Run circuit
final_state = run_program(my_qpu, my_circuit, { "global_1": 3.1415, "global_2": 1.5708 })

print(np.round(final_state))

[ 0.+0.j  0.+0.j -0.+1.j  0.+0.j]
